# Sentry Demo
Welcome to the interactive demonstration of Aleno's Sentry API, crafted specifically for DeFi Asset Managers. This notebook is designed to guide you through the powerful features of Sentry, an advanced alerting system that enables meticulous monitoring of financial risks across DeFi platforms.

#### Documentation
For detailed information about all available API routes and features, please consult our official [documentation](https://docs.aleno.ai).

Objective
In this demonstration, you'll learn:

- How to configure simple alerts using Sentry
- How to visualize the recent metrics variations
- How to set up Sentry to send customized alerts directly to your Telegram, integrated via [Zapier](https://zapier.com/).
By the end of this notebook, you'll be able to customize Sentry alerts to fit your DeFi portfolio needs, improving your asset management and risk reduction strategies.

Let’s get started!

**NOTE**: For this notebook, it's assumed you have already obtained an API_KEY. Please ensure you replace it in the following cell.


In [ ]:
import requests 

API_KEY = "YOUR_API_KEY"

headers = { 
    'Authorization': API_KEY,
    'Content-Type': 'application/json'
}

# This demo will be done on ETH chain only
CHAIN_ID = "eth"
sentry_base_url = "https://sentry.aleno.ai"

 # 1. Create a new user

 Sentry's design allows to separate the API usage in many User for a single API key. This allow to create several desktop for a single organization. If you are the single user of the API you will create a single user during first use.

In [ ]:

data = { "users" : 
    # Our list has only one element as we create one unique user
    [{
        # Optional field a username to identify the user
        "userName": "Alice", 
        # Optional field a user context any string that can allow to identify your user in your internal business logic
        "userContext": "alice_context"
    }]
}


user = requests.post(
    f'{sentry_base_url}/users',
    json=data,
    headers=headers
).json()['data'][0]

print(f'User created with id:{user["id"]}')

 # 2. Find meaningful metrics

Now that we have an Api key and a user we are ready to subscribe to metrics in order to receive Alerts.

#### Metrics Overview

Currently Sentry Api supports a set 6 types of metrics that will rapidly grow. Metrics can be split into 2 types.

- **Token metrics**: These metrics allows to track the activity of a given asset across DeFi.
- **Pool metrics**: These metrics allows to monitor the activity of an AMM's pool. Most of AMM are currently supported.

Currently supported metrics are:

Token metrics:
- **Token Total TVL**: This metric represent the total amount of liquid asset in AMMs pools. Alerts are triggered as soon as the aggregated TVL of all pools of a token highly increases/decreases in a block or a user-defined time window.
- **Token Total Supply**: Alerts are triggered as soon as the total supply of a given token highly increases/decreases in a block or a user-defined time window. This is useful to monitor LRT/LST total deposited amount as the total supply of an LRT is equal to the amount of ETH deposited
- **Token USD price**: Alerts are triggered as soon as the USD price of a token highly increases/decreases in a block or a user-defined time window.
- **Token WETH price**: Alerts are triggered as soon as the WETH price of a token highly increases/decreases in a block or a user-defined time window. 
 
 Pool metrics:
 - **Pool TVL**:  Alerts are triggered as soon as the TVL of a pool highly increases/decreases in a block or a user-defined time window. This metric can allow to monitor pool draining or large token dumps.
 - **Pool Rates**:  Alerts are triggered as soon as the rate of a pool highly increases/decreases in a block or a user-defined time window.

 A complete list of upcoming metrics can be found in our official [documentation](https://docs.aleno.ai).



Within Sentry, each metric can be identified by its **key**. There are several methods to find a relevant metric to subscribe to, including recreating the key based on known parameters or using suggestion routes.

### A. Reconstruct a metric key

A metric key is in the form of:`chainId_metricType_parameters`. By replacing the ``{address}` parameter in the key, you could retrieve any metric if it is supported.

- **Token TVL**: eth_token_total_tvl_{tokenAddress}
- **Token Supply**: eth_token_total_supply_{tokenAddress}
- **Token Weth**: eth_token_weth_price_{tokenAddress}
- **Token USD price**: eth_token_usd_price_{tokenAddress}
- **Pool TVL**: eth_pool_tvl_{poolAddress}_{tokenAddress}
- **Pool Rate**: eth_pool_rate_{poolAddress}_{baseTokenAddress}_{quoteTokenAddress}

Let's check if the ezETH price in WETH is supported

In [ ]:
# Careful, all addresses should be lowercase !!
ezETH_token_address = '0xbf5495efe5db9ce00f80364c8b423567e58d2110'
metric_key = f'eth_token_weth_price_{ezETH_token_address}'

# Make a GET request, passing the list of token addresses as a 'token_address' parameter
response = requests.get(
    f'{sentry_base_url}/metrics',
    params={'keys':  metric_key,
            'chainId': 'eth'}
)

# Convert the response to JSON and access the 'data' part
ezETH_price_metric = response.json()['data']
print(ezETH_price_metric)

We see that we are getting the metric result with all its info, so the crafted metrics is supported. We can also display its recent evolution using the metrics/records/range route.

In [ ]:
import pandas as pd
import plotly.express as px

response = requests.get(
    f'{sentry_base_url}/metrics/records/range',
    params={'key':  metric_key},
    headers=headers
)

# Convert the response to JSON and access the 'data' part
ezETH_price_history = response.json()['data']

df = pd.DataFrame(ezETH_price_history['points'])
df['date'] = pd.to_datetime(df['timestamp'],unit='s')

# Creating the line plot
fig = px.line(df, x='date', y='value', title='Line Plot of Date vs Value', labels={'Value': 'Price'})
fig.show()

### B. Find metrics using tokens/pools addresses

Instead of manually building the keys one by one, it could be more convenient to retrieve the full set of keys for given tokens/pools.

Lets see an example by fetching all metrics related to the ezETH token address

In [ ]:
# Note that one could fetch multiple addresses at once. For the simplicity of this demo we only consider one token
token_addresses = [ezETH_token_address]

response = requests.get(
    f'{sentry_base_url}/metrics',
    params={'tokenAddresses':  ','.join(token_addresses),
            'chainId': 'eth'}
)

# Convert the response to JSON and access the 'data' part
metrics = response.json()['data']
print(f'{len(metrics)} metrics found')
print([metric['key'] for metric in metrics])

### C. Find metrics using wallet addresses

To get a list of metrics suggestion for a given portfolio (list of wallet addresses), you can get the /suggestion route. This route will suggest you all supported metrics based on assets a nd positions found in you wallet. It also returns the list of supported/unsupported assets that were found in the portfolio.

Don't hesitate to reach out if you find major unsupported assets we will add them ASAP.

In [ ]:
# Note that one could fetch multiple addresses at once. For the simplicity of this demo we only consider one token
wallet_addresses = ["0x30dff99282b36b93f96376fd565c981e013289a0".lower()]

response = requests.get(
    f'{sentry_base_url}/suggestions',
    params={'addresses':  ','.join(wallet_addresses),
            # only get suggestions for held assets worth more than 100$
            "assetUsdThreshold": 100,
            "positionUsdThreshold": 100,
            'chainId': 'eth'
            },
    headers=headers
)

# Convert the response to JSON and access the 'data' part
suggestions = response.json()['data']
suggested_metrics_keys = [metric['key'] for metric in suggestions['metrics']]
suggested_metrics_keys

# 3. Subscribe to metrics

Now that we've found interesting metrics let's actually subscribe to them for the user created in step 1.

In this example we will subscribe to all metrics suggested for the wallet address provided above. We set small thresholds in order to test the notifications. 

In [ ]:
# Here we assign the same threshold to all users, but this could be adjusted depending on you logic
subscription_data = [{ 
    'userId': user['id'],
    'threshold': 0.01,
    'metricKey': key  
    } for key in suggested_metrics_keys]

response = requests.post(
    f'{sentry_base_url}/subscriptions',
    json= { "subscriptions": subscription_data },
    headers=headers
)

print(f'Successfully updated to {len(response.json()["data"]['upsertedSubscriptions'])} metrics')

# 4. Receive the notifications

### a. Receive real time alerts

Currently, alerts can only be sent to a webhook or to telegram. You have 3 choices to receive notifications:

- setup your own webhook server and handle notifications with you custom logic
- setup a zapier account and create a Zap to process the notifications and send them to your preferred device. Everything is explained in the official with an example with telegram [documentation](https://docs.aleno.ai/use-cases/alerts-on-telegram-with-zapier)
- setup a telegram channel and configure sentry to send notifications to that channel as explained in the [documentation](https://docs.aleno.ai/use-cases/alerts-on-telegram-with-zapier)

Here we assume that you've configured a telegram channel and know the channel ID to be replaced in YOUR_TELEGRAM_ID 

In [ ]:
telegram_channel_id = '-4220052563'
response = requests.put(
    f'{sentry_base_url}/account',
    json={'telegramChannelId':telegram_channel_id},
    headers=headers 
).json()

print(response['data'])

You can now see alerts coming in your telegram channel.

### b. Check alert history

Alerts can also be checked by regularly polling the alert history endpoint as follow.

In [ ]:
response = requests.get(
    f'{sentry_base_url}/alertHistory',
    # Optional get alerts for the created user
    params={'userId': user['id'], 'pageSize':100},
    headers=headers
).json()

alerts_df = pd.DataFrame(response['data'])
alerts_df

# 5. Drop subscriptions 

Here we show how to fetch all metrics and delete them. To delete a subscription, just update it's threshold to 0.

In [ ]:
for sub in subscription_data:
    sub['threshold'] = 0

number_deleted_subscriptions = requests.post(
    f'{sentry_base_url}/subscriptions',
    json= { "subscriptions": subscription_data },
    headers=headers
).json()['data']['numberDeletedSubscription']

print(f'Successfully deleted {number_deleted_subscriptions} subscriptions')